## Autonomous Planner Agent Implementation

In [2]:
import json
from dotenv import load_dotenv
import logging
from openai import OpenAI
import chromadb

from agents.scanner_agent import ScannerAgent

In [3]:
load_dotenv(override=True)
openai = OpenAI() 
MODEL = "gpt-5-mini"

## Start with some test data and test agentic function/tool calling

In [4]:
test_results = ScannerAgent().test_scan()
test_results

DealSelection(deals=[Deal(product_description="The Hisense R6 Series 55R6030N is a 55-inch 4K UHD Roku Smart TV that offers stunning picture quality with 3840x2160 resolution. It features Dolby Vision HDR and HDR10 compatibility, ensuring a vibrant and dynamic viewing experience. The TV runs on Roku's operating system, allowing easy access to streaming services and voice control compatibility with Google Assistant and Alexa. With three HDMI ports available, connecting multiple devices is simple and efficient.", price=178.0, url='https://www.dealnews.com/products/Hisense/Hisense-R6-Series-55-R6030-N-55-4-K-UHD-Roku-Smart-TV/484824.html?iref=rss-c142'), Deal(product_description='The Poly Studio P21 is a 21.5-inch LED personal meeting display designed specifically for remote work and video conferencing. With a native resolution of 1080p, it provides crystal-clear video quality, featuring a privacy shutter and stereo speakers. This display includes a 1080p webcam with manual pan, tilt, and

In [5]:
### Mockup/Simulation function
def scan_the_internet_for_bargains() -> str: 
    """ This tool scans the internet for great deals and gets a curated list of promising deals """
    print("Fake function to scan the internet -- this returns a hardcoded set of deals.")
    return test_results.model_dump_json()

In [6]:
### Mockup/Simulation function 
def estimate_true_value(description: str) -> str:
    """
    This tool estimates the true value of a product based on a text description of it
    """
    print(f"Fake function to estimating true value of {description[:20]}... - this always returns $300") 
    return f"Product {description} has an estimated true value of $300"

In [7]:
def notify_user_of_deal(description: str, deal_price: float, estimated_true_value: float, url: str) -> str:
    """
    This tool notifies the user of a great deal, given a description of it, the price of the deal, and the estimated true value
    """
    print(f"Fake function to notify user of {description} which costs {deal_price} and estimate is {estimated_true_value}")
    return "notification has been sent"

In [8]:
notify_user_of_deal("a new iphone", 100, 1000, "https://www.apple.com/iphone")

Fake function to notify user of a new iphone which costs 100 and estimate is 1000


'notification has been sent'

## Schema Definition 
Build Json schema structure to describe tools to the LLM model.

In [9]:
scan_function = {
        "name": "scan_the_internet_for_bargains",
        "description": "Returns top bargains scraped from the internet along with the price each item is being offered for",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
            "additionalProperties": False
        }
    }

estimate_function = {
    "name": "estimate_true_value",
    "description": "Given the description of an item, estimate how much it is actually worth",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item to be estimated"
            },
        },
        "required": ["description"],
        "additionalProperties": False
    }
}

notify_function = {
    "name": "notify_user_of_deal",
    "description": "Send the user a push notification about the single most compelling deal; only call this one time",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item itself scraped from the internet"
            },
            "deal_price": {
                "type": "number",
                "description": "The price offered by this deal scraped from the internet"
            }
            ,
            "estimated_true_value": {
                "type": "number",
                "description": "The estimated actual value that this is worth"
            }
            ,
            "url": {
                "type": "string",
                "description": "The URL of this deal as scraped from the internet"
            }
        },
        "required": ["description", "deal_price", "estimated_true_value", "url"],
        "additionalProperties": False
    }
}

In [10]:
tools = [
    {"type": "function", "function": scan_function}, 
    {"type": "function", "function": estimate_function}, 
    {"type": "function", "function": notify_function}
]

In [11]:
tools

[{'type': 'function',
  'function': {'name': 'scan_the_internet_for_bargains',
   'description': 'Returns top bargains scraped from the internet along with the price each item is being offered for',
   'parameters': {'type': 'object',
    'properties': {},
    'required': [],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'estimate_true_value',
   'description': 'Given the description of an item, estimate how much it is actually worth',
   'parameters': {'type': 'object',
    'properties': {'description': {'type': 'string',
      'description': 'The description of the item to be estimated'}},
    'required': ['description'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'notify_user_of_deal',
   'description': 'Send the user a push notification about the single most compelling deal; only call this one time',
   'parameters': {'type': 'object',
    'properties': {'description': {'type': 'string',
      'description

In [31]:
system_message = "You find great deals on bargain products using your tools, and notify the user of the best bargain."
user_message = """
First, use your tool to scan the internet for bargain deals. Then for each deal, use your tool to estimate its true value.
Then pick the single most compelling deal where the price is much lower than the estimated true value, and use your tool to notify the user.
Then just reply OK to indicate success.
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

In [28]:
messages

[{'role': 'system',
  'content': 'You find great deals on bargain products using your tools, and notify the user of the best bargain.'},
 {'role': 'user',
  'content': '\nFirst, use your tool to scan the internet for bargain deals. Then for each deal, use your tool to estimate its true value.\nThen pick the single most compelling deal where the price is much lower than the estimated true value, and use your tool to notify the user.\nThen just reply OK buddy to indicate success.\n'}]

In [29]:
def handle_tool_call(message):
    """
    Call the tools associated with this message
    """
    results = []

    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        ### access any corresponding function dynamically (with no explicit func name matching)
        tool = globals().get(tool_name) 
        # print("TOOL: ", tool)
        ### result is what the corresponding/chosen function returns
        result = tool(**arguments) if tool else {}
        results.append(
            {"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id}
        )

    # print("RESULTS: ", results)

    return results

In [25]:
### Append the tool call response into the messages list so LLM continues to process it
while True:
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    base_path = response.choices[0]
    if base_path.finish_reason == "tool_calls": 
        message = base_path.message
        results = handle_tool_call(message)
        messages.append(message) # assistant's tool call message with the tool_calls property
        messages.extend(results) # tool response
        continue
    else: 
        break

response.choices[0].message.content

Fake function to scan the internet -- this returns a hardcoded set of deals.
Fake function to estimating true value of Hisense R6 Series 55... - this always returns $300
Fake function to estimating true value of Poly Studio P21 21.5... - this always returns $300
Fake function to estimating true value of Lenovo IdeaPad Slim ... - this always returns $300
Fake function to estimating true value of Dell G15 gaming lapt... - this always returns $300
Fake function to notify user of The Hisense R6 Series 55R6030N is a 55-inch 4K UHD Roku Smart TV that offers stunning picture quality with 3840x2160 resolution. It features Dolby Vision HDR and HDR10 compatibility, ensuring a vibrant and dynamic viewing experience. The TV runs on Roku's operating system, allowing easy access to streaming services and voice control compatibility with Google Assistant and Alexa. With three HDMI ports available, connecting multiple devices is simple and efficient. which costs 178 and estimate is 300


'OK buddy'